In [ ]:
''' 
Result Visualization
Author: Yitian Shao
Created on 2022.01.20
Merge adpated on 2022.01.28
Updated on 2022.03.04 for new measurement
'''
# %matplotlib notebook 
# %matplotlib notebook 

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

plt.rc('font', size=18, family='Verdana') # 'Tahoma', 'DejaVu Sans', 'Verdana'
plt.rc('axes', edgecolor='0.5', linewidth=0.75)
plt.rc('axes.spines', **{'bottom':True, 'left':True, 'right':False, 'top':False})

In [ ]:
''' General functions '''
def selectConditions(dFrame, condiList):
    #ind = np.zeros(dFrame['CondiName'].shape, dtype=bool)
    indList = []
    for aCondi in condiList:
        #ind = ind | (dFrame['CondiName'] == aCondi)
        indList.extend(dFrame[dFrame['CondiName'] == aCondi].index)
        
    return indList

In [ ]:
''' Load data '''
condiTable = pd.read_csv("./data./ConditionTable.csv") # Table of measurement conditions

measureData = pd.read_csv("./data./Measurements.csv") # Data from physical measurements

measureData.tail()

In [ ]:
# simData = pd.read_csv("./data./temp_result.csv") # Data from model simulation

# ''' Average and median rising time of each condition '''

# simData['deltaT'] = np.nan
# simData['deltaTm'] = np.nan

# trialsData = []
# trialMax = []

# for aCondi in condiTable['CondiName']:
#     allTrials = measureData[measureData['CondiName'] == aCondi]
#     avgDeltaT = np.mean(allTrials['RiseTime_sec'])
#     medDeltaT = np.median(allTrials['RiseTime_sec'])
#     print("%s - average deltaT = %.3f sec, median deltaT = %.3f sec" % (aCondi, avgDeltaT, medDeltaT))
    
#     simData.loc[simData['CondiName'] == aCondi,'deltaT'] = avgDeltaT
# #     simData.loc[simData['CondiName'] == aCondi,'deltaTm'] = medDeltaT

#     trialsData.append(allTrials['AvgPressure_kPa'])
#     trialMax.append(allTrials['AvgPressure_kPa'].max())
    
# simData['pLossFull'] = simData['lossFactor']*simData['dVFull_m3']/simData['deltaT']
# simData['pLoss1Expan'] = simData['lossFactor']*simData['dV1Expan_m3']/simData['deltaT']

# # simData['pLoss1ExpanM'] = simData['lossFactor']*simData['dV1Expan_m3']/simData['deltaTm']



In [ ]:
''' Pressure loss from realtime flow rate estimation '''

simData = pd.read_csv("./data/temp_result.csv") # Data from model simulation
# simData['pLoss1Expan_kPa'] = np.nan

for aCondi in condiTable['CondiName']:
    allTrials = measureData[measureData['CondiName'] == aCondi]
    simData.loc[simData['CondiName'] == aCondi,'pLoss1Expan_kPa'] = np.mean(allTrials['P1ExpanLoss_kPa'])


fig1, ax1 = plt.subplots()
fig1.set_size_inches(18,6)
fig1.suptitle("Avg. Pressure of Simulation (Orange) vs. Measurement (Red)")
ax1.set_xlabel('Condition')
ax1.set_xticklabels(simData['CondiName'], rotation=80)

ax1.plot(simData['CondiName'], simData['P1Expan_Pa']*1e-3, '-', color='k')
ax1.plot(simData['CondiName'], simData['P1Expan_Pa']*1e-3+simData['pLoss1Expan_kPa'], 's--', color='tab:orange')

ax1.plot(measureData['CondiName'], measureData['AvgPressure_kPa'], '.',color='tab:red')
ax1.set_ylabel('Pressure (kPa)', color='tab:orange')

In [ ]:
''' Compare material '''
compList = ['BOPP16ShiftedU7kV','L0Ws16ShiftedU9kV','Mylar16'] # List of conditions to be compared
compList2 = ['BOPP16','L0Ws16NoShift','Mylar16NoShift'] # List of conditions (Dash No shift) to be compared

indList = selectConditions(simData, compList)
indList2 = selectConditions(simData, compList2)
print(simData.loc[indList,'CondiName']); print(simData.loc[indList2,'CondiName'])

# sim1 = np.array(simData.loc[indList,'P1Expan_Pa'] - simData.loc[indList,'pLoss1Expan'])*1e-3 # constant estimation
# sim2 = np.array(simData.loc[indList2,'P1Expan_Pa'] - simData.loc[indList2,'pLoss1Expan'])*1e-3 # constant estimation

sim1 = np.array(simData.loc[indList,'P1Expan_Pa']*1e-3 + simData.loc[indList,'pLoss1Expan_kPa']) # Realtime estimation
sim2 = np.array(simData.loc[indList2,'P1Expan_Pa']*1e-3 + simData.loc[indList2,'pLoss1Expan_kPa']) # Realtime estimation

simAvg = 0.5 * (sim1 + sim2)

trialsData = []
trialsDataMax = []
for aPair in zip(compList, compList2): # Ordered based on the list
    mergedCondi = measureData.loc[measureData['CondiName'] == aPair[0], 'AvgPressure_kPa'].tolist()
    mergedCondi.extend(measureData.loc[measureData['CondiName'] == aPair[1], 'AvgPressure_kPa'].tolist())
    trialsData.append(mergedCondi)    
    
    mergedCondi = measureData.loc[measureData['CondiName'] == aPair[0], 'MaxPressure_kPa'].tolist()
    mergedCondi.extend(measureData.loc[measureData['CondiName'] == aPair[1], 'MaxPressure_kPa'].tolist())
    trialsDataMax.append(np.array(mergedCondi) ) 

trialsDataMax = np.array(trialsDataMax)
trialsDataMaxMean = np.max(trialsDataMax, axis = 1)
    
''' Generate plot '''    
boxprops = dict(linestyle='-', linewidth=1, color='w', facecolor='0.85')
medianprops = dict(linestyle='-', linewidth=3, color='tab:orange')
    
fig1, ax1 = plt.subplots(dpi=300)
fig1.set_size_inches(6,6)
#fig1.suptitle("Avg. Pressure provide by different shell materials")
ax1.set_xlabel('Shell material')
ax1.set_xticklabels(compList, rotation=0)

x = range(len(compList))

pH = ax1.plot(x, simAvg, '-', color='k')

pH2 = ax1.boxplot(trialsData, positions=x, widths = 0.15, manage_ticks = False, medianprops=medianprops, 
                 patch_artist=True, boxprops=boxprops, notch = False, showfliers = True)

# pH3 = ax1.violinplot(trialsDataMax, positions=x, showmeans=False, showmedians=False, showextrema=False)
pH3 = ax1.plot(trialsDataMaxMean, 's')

ax1.set_ylabel('Avg. Pressure (kPa)', color='k')
ax1.tick_params(axis='y', labelcolor='k')

plt.xticks(x, condiTable.loc[indList, 'Shell']);

# ax1.legend(handles=[pH2['medians'][0],pH[0]],labels=['Meas. Median','Model'], frameon=False);
# plt.savefig("comp_shell_material.pdf", bbox_inches='tight')

# ax1.legend(handles=[pH2['medians'][0],pH[0],pH3['bodies'][0]],labels=['Meas. Median (Time-avg.)','Model (Time-avg.)','Meas. (Max)'], 
#            frameon=False, bbox_to_anchor=(0.9, 1.0), loc='upper left');
ax1.legend(handles=[pH2['medians'][0],pH[0],pH3[0]], labels=['Meas. Median (Time-avg.)','Model (Time-avg.)','Meas. (Max. Peak)'], 
           frameon=False, bbox_to_anchor=(1.02, 1.0), loc='upper left');

plt.savefig("TriPlot.pdf", bbox_inches='tight')

In [ ]:
''' Compare voltage (L0Ws - Temporary) '''
# compList = ['L0WsSi5-20Trek8kV','L0WsSi5-20Trek9kV','L0WsSi5-20Trek10kV'] # List of conditions to be compared
# compList = ['L0WsSi6-16Trek9kVHVTop','L0WsSi6-16Trek10kVHVTop'] # List of conditions to be compared
# compList = ['MLSi20HV7kV.5Hz','MLSi20HV7kV1Hz'] #''' Twenty measurements (2022.04.02) '''
compList = ['MLSi20HV7kV6mL.5Hz','MLSi20HV7kV7mL.5Hz','MLSi20HV7kV8mL.5Hz','MLSi20HV7kV10mL.5Hz'] #''' Twenty measurements (2022.04.02) '''

indList = selectConditions(simData, compList)
print(simData.loc[indList,'CondiName']);

sim1 = np.array(simData.loc[indList,'P1Expan_Pa']*1e-3) # Realtime estimation  + simData.loc[indList,'pLoss1Expan_kPa']

trialsData = [] # Time-avg. Pressure
trialsDataMaxMean = [] # Max. Pressure 
for aCondi in (compList): # Ordered based on the list
    aTrial = measureData.loc[measureData['CondiName'] == aCondi, 'AvgPressure_kPa'].to_numpy()
    trialsData.append(aTrial)   
    print("%s - %d trials" % (aCondi, len(aTrial)))
    
    trialsDataMaxMean.append(np.max(measureData.loc[measureData['CondiName'] == aCondi, 'MaxPressure_kPa'].to_numpy()))  

''' Generate plot '''    
boxprops = dict(linestyle='-', linewidth=1, color='w', facecolor='0.85')
medianprops = dict(linestyle='-', linewidth=3, color='tab:orange')
    
fig1, ax1 = plt.subplots(dpi=300)
fig1.set_size_inches(6,6)

# ax1.set_xlabel('Voltage (kV)')
# ax1.set_xlabel('Condition') # For showing conditions of twenty trials

x = range(len(compList))

ax1.set_ylabel('Avg. Pressure (kPa)', color='k')
ax1.tick_params(axis='y', labelcolor='k')

# pH = ax1.plot(x, sim1, '--', color='k')
pH = ax1.plot(x, sim1, '-', color='k')

pH2 = ax1.boxplot(trialsData, positions=x, widths = 0.15, manage_ticks = False, medianprops=medianprops, 
                 patch_artist=True, boxprops=boxprops, notch = False, showfliers = True)

# pH3 = ax1.violinplot(trialsDataMax, positions=x, showmeans=False, showmedians=False, showextrema=False)# Violin
pH3 = ax1.plot(trialsDataMaxMean, 's')

# ax1.set_xticklabels(compList, rotation=0)
# plt.xticks(x, 0.001 * condiTable.loc[indList, 'Voltage_V']);

# ax1.set_xticklabels(['6mL 0.5Hz','6mL 1Hz'], rotation=0) #''' Twenty measurements (Only for two cases 2022.04.02) '''
# plt.xticks([0, 1]) # For showing conditions of twenty trials

ax1.set_xticklabels([re.findall("\d+mL", nameStr)[0] for nameStr in compList], rotation=0) # (Only for compare volume 2022.04.02) 
plt.xticks(range(len(compList)))
ax1.set_xlabel('Infill Volume')

# ax1.legend(handles=[pH2['medians'][0],pH[0],pH3['bodies'][0]],labels=['Meas. Median (Time-avg.)','Model (Time-avg.)','Meas. (Max)'], 
#            frameon=False, bbox_to_anchor=(0.9, 1.0), loc='upper left');
ax1.legend(handles=[pH2['medians'][0],pH[0],pH3[0]], labels=['Meas. Median (Time-avg.)','Model (Time-avg.)','Meas. (Max. Peak)'], 
           frameon=False, bbox_to_anchor=(0.9, 1.0), loc='upper left');

plt.savefig("TriPlot.pdf", bbox_inches='tight')

In [ ]:
''' Compare Frequency '''
compList = ['MLSi20HV7kV10mL.5Hz',
            'MLSi20HV7kV10mL1Hz',
            'MLSi20HV7kV10mL2Hz',
            'MLSi20HV7kV10mL4Hz',
            'MLSi20HV7kV10mL10Hz',
            'MLSi20HV7kV10mL20Hz',
            ] #''' Twenty measurements (2022.04.02) '''
xLabelStr = [re.findall('[\.]*\d+Hz', aName)[0] for aName in compList]

indList = selectConditions(simData, compList)
print(simData.loc[indList,'CondiName']);

# sim1 = np.array(simData.loc[indList,'P1Expan_Pa']*1e-3 + simData.loc[indList,'pLoss1Expan_kPa']) # Realtime estimation incorrect tube 
sim1 = np.array(simData.loc[indList,'P1Expan_Pa']*1e-3) # Tube fluid friction loss model not working

trialsData = []
trialsDataMax = [] # Max. Pressure 
trialsDataMaxMean = []
for aCondi in (compList): # Ordered based on the list
    aTrial = measureData.loc[measureData['CondiName'] == aCondi, 'AvgPressure_kPa'].tolist()
    trialsData.append(aTrial)   
    print("%s - %d trials" % (aCondi, len(aTrial)))
    
    trialsDataMax.append(measureData.loc[measureData['CondiName'] == aCondi, 'MaxPressure_kPa'].tolist())  
    
    trialsDataMaxMean.append(np.max(measureData.loc[measureData['CondiName'] == aCondi, 'MaxPressure_kPa'].to_numpy())) 
    
''' Generate plot '''    
boxprops = dict(linestyle='-', linewidth=1, color='w', facecolor='0.85')
medianprops = dict(linestyle='-', linewidth=3, color='tab:orange')
    
fig1, ax1 = plt.subplots(dpi=300)
fig1.set_size_inches(6,6)

ax1.set_xlabel('Condition') # For showing conditions of twenty trials

x = range(len(compList))

pH = ax1.plot(x, sim1, '--', color='k')

pH2 = ax1.boxplot(trialsData, positions=x, widths = 0.15, manage_ticks = False, medianprops=medianprops, 
                 patch_artist=True, boxprops=boxprops, notch = False, showfliers = True)

# pH3 = ax1.violinplot(trialsDataMax, positions=x, showmeans=False, showmedians=False, showextrema=False)
pH3 = ax1.plot(trialsDataMaxMean, 's')

ax1.set_ylabel('Avg. Pressure (kPa)', color='k')
ax1.tick_params(axis='y', labelcolor='k')

ax1.set_xticklabels(xLabelStr, rotation=0)
plt.xticks(x) # For showing conditions of twenty trials

# ax1.legend(handles=[pH2['medians'][0],pH[0]],labels=['Meas. Median','Model'], frameon=False, loc='best');
# ax1.legend(handles=[pH2['medians'][0],pH[0],pH3['bodies'][0]],labels=['Meas. Median (Time-avg.)','Model (Time-avg.)','Meas. (Max)'], 
#            frameon=False, bbox_to_anchor=(1.0, 1.0), loc='upper left');
ax1.legend(handles=[pH2['medians'][0],pH[0],pH3[0]], labels=['Meas. Median (Time-avg.)','Model (Time-avg.)','Meas. (Max. Peak)'], 
           frameon=False, bbox_to_anchor=(1.02, 1.0), loc='upper left');

plt.savefig("TriPlot.pdf", bbox_inches='tight')


In [ ]:
''' Obsoleted Analysis '''
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(16,6)
# fig1.suptitle("Avg. Pressure of Simulation (Orange) vs. Measurement (Red)")
# ax1.set_xlabel('Condition')
# ax1.set_xticklabels(simData['CondiName'], rotation=80)

# ax1.plot(simData['CondiName'], simData['P1Expan_Pa']*1e-3, '--', color='k')
# ax1.plot(simData['CondiName'], (simData['P1Expan_Pa'] - simData['pLoss1Expan'])*1e-3, '-', color='tab:orange')
# # ax1.plot(simData['CondiName'], (simData['PFull_Pa'] - simData['pLossFull'])*1e-3, '--', color='b')
# #ax1.plot(simData['CondiName'], (simData['P1Expan_Pa'] - simData['pLoss1ExpanM'])*1e-3, color='b')
# ax1.plot(measureData['CondiName'], measureData['AvgPressure_kPa'], '.',color='tab:red')
# # ax1.plot(measureData['CondiName'], measureData['MaxPressure_kPa'], '.',color='tab:blue')
# ax1.set_ylabel('Pressure (kPa)', color='tab:orange')
# ax1.tick_params(axis='y', labelcolor='tab:orange')

# # ax1.set_ylim([0, 22.5]);

''' Boxplot of trials (Obsoleted)'''
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(16,6)
# fig1.suptitle("Avg. Pressure of Simulation (Orange) vs. Measurement (Red)")
# ax1.set_xlabel('Condition')
# ax1.set_xticklabels(simData['CondiName'], rotation=80)

# x = range(len(simData))

# ax1.plot(x, trialMax, '--', color='tab:grey')

# ax1.plot(x, (simData['P1Expan_Pa'] - simData['pLoss1Expan'])*1e-3, '-', color='tab:orange')
# box1 = ax1.boxplot(trialsData, positions=x, widths = 0.3, manage_ticks = False, medianprops=dict(color='red'))

# ax1.set_ylabel('Pressure (kPa)', color='tab:orange')
# ax1.tick_params(axis='y', labelcolor='tab:orange')

# plt.xticks(x, simData['CondiName'])

# ax1.set_ylim([0, 22.5]);

''' Obsoleted Analysis '''
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(16,6)
# fig1.suptitle("Avg. Pressure of Simulation (Orange) vs. Peak Pressure of Measurement (blue)")
# ax1.set_xlabel('Condition')
# ax1.set_xticklabels(simData['CondiName'], rotation=80)

# ax1.plot(simData['CondiName'], (simData['P1Expan_Pa'] - simData['pLoss1Expan'])*1e-3, '-', color='tab:orange')
# ax1.plot(measureData['CondiName'], measureData['MaxPressure_kPa'], '.',color='tab:blue')
# ax1.set_ylabel('Pressure (kPa)', color='tab:orange')
# ax1.tick_params(axis='y', labelcolor='tab:orange')

# ax1.set_ylim([0, 25]);

''' Obsoleted Analysis '''
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(16,8)
# fig1.suptitle("Pressure (Orange) vs. Driven Voltage (Blue)")
# ax1.set_xlabel('Condition')
# ax1.set_xticklabels(simData['CondiName'], rotation=80)

# ax1.plot(simData['CondiName'], simData['PFull_Pa']*1e-3, color='tab:orange')
# ax1.plot(measureData['CondiName'], measureData['AvgPressure_kPa'], '.',color='tab:red')
# ax1.set_ylabel('Pressure (kPa)', color='tab:orange')
# ax1.tick_params(axis='y', labelcolor='tab:orange')

# ax2 = ax1.twinx() 
# ax2.plot(condiTable['CondiName'], condiTable['Voltage_V']*1e-3, '-', color='tab:blue')
# ax2.set_ylabel('Voltage (kV)', color='tab:blue')
# ax2.tick_params(axis='y', labelcolor='tab:blue')

# fig1.tight_layout() 

# ax2.set_ylim([5, 10]);

''' Obsoleted Analysis '''
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(16,10)
# fig1.suptitle("Pressure (Orange) vs. Material of Shell (Blue x) and Fluid (Black squre)")
# ax1.set_xlabel('Condition')
# ax1.set_xticklabels(simData['CondiName'], rotation=80)

# ax1.plot(simData['CondiName'], simData['PFull_Pa']*1e-3, color='tab:orange')
# ax1.plot(measureData['CondiName'], measureData['AvgPressure_kPa'], '.',color='tab:red')
# ax1.set_ylabel('Pressure (kPa)', color='tab:orange')
# ax1.tick_params(axis='y', labelcolor='tab:orange')

# ax2 = ax1.twinx() 
# ax2.plot(condiTable['CondiName'], condiTable['Shell'], 'x',color='tab:blue')
# ax2.plot(condiTable['CondiName'], condiTable['Fluid'], 's',color='k')
# ax2.set_ylabel('Material', color='tab:blue')
# ax2.tick_params(axis='y', labelcolor='tab:blue')

# fig1.tight_layout() 

# ax2.set_ylim([-4, 5]);

''' Obsoleted Analysis '''
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(10,6)
# fig1.suptitle("Pressure (Orange) vs. Layout (Blue)")
# ax1.set_xlabel('Condition')
# ax1.set_xticklabels(simData['CondiName'], rotation=80)

# ax1.plot(simData['CondiName'], simData['PFull']*1e-3, color='tab:orange')
# ax1.plot(measureData['CondiName'], measureData['AvgPressure_kPa'], '.',color='tab:red')
# ax1.set_ylabel('Pressure (kPa)', color='tab:orange')
# ax1.tick_params(axis='y', labelcolor='tab:orange')

# ax2 = ax1.twinx() 
# ax2.plot(condiTable['CondiName'], condiTable['Layout'], 'x',color='tab:blue')
# ax2.set_ylabel('Layout', color='tab:blue')
# ax2.tick_params(axis='y', labelcolor='tab:blue')

# fig1.tight_layout() 

# ax2.set_ylim([-6, 6]);

''' Obsoleted Analysis '''
# fig1, ax1 = plt.subplots()
# fig1.set_size_inches(10,6)
# fig1.suptitle("Pressure (Orange) vs. Row Space (Blue x) and Dash length (Black square)")
# ax1.set_xlabel('Condition')
# ax1.set_xticklabels(simData['CondiName'], rotation=80)

# ax1.plot(simData['CondiName'], simData['PFull']*1e-3, color='tab:orange')
# ax1.plot(measureData['CondiName'], measureData['AvgPressure_kPa'], '.',color='tab:red')
# ax1.set_ylabel('Pressure (kPa)', color='tab:orange')
# ax1.tick_params(axis='y', labelcolor='tab:orange')

# ax2 = ax1.twinx() 
# ax2.plot(condiTable['CondiName'], condiTable['DashSpace_mm'], 'x',color='tab:blue')
# ax2.plot(condiTable['CondiName'], condiTable['DashLength_mm'], 's',color='k')
# ax2.set_ylabel('(mm)', color='tab:blue')
# ax2.tick_params(axis='y', labelcolor='tab:blue')

# fig1.tight_layout() 

# ax2.set_ylim([0, 21]);